In [2]:
import os
import tensorflow as tf

In [3]:
tf_record_filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./test_record.tfrecord"))
#f=open("./test_record.tfrecord",'rb')
tf_record_reader=tf.TFRecordReader()
_, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

In [15]:
tf_record_features = tf.parse_single_example(
    tf_record_serialized,
    features={
        'label': tf.FixedLenFeature([], tf.string),
        'width': tf.FixedLenFeature([], tf.int64),
        'height': tf.FixedLenFeature([], tf.int64),
        'image': tf.FixedLenFeature([], tf.string),
    })

In [45]:
tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)

tf_record_image = tf.reshape(tf_record_image, [64,64,1])

tf_record_label = tf.cast(tf_record_features['label'],tf.string)

init=tf.local_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    img, lab = sess.run([tf_record_image,tf_record_label])
    print(len(img[0]))
    
    coord.request_stop()
    coord.join(threads)



64
